# Determining the Path Tortuosity

The tortuosity of a path is a descriptor of how twisted the path is - how many turns it has and how restricted those are, or if the curve is disconnected. This outputs an arbitrary positive value quantifying the tortuosity, where close to zero is free space and higher values describe a medium with closed and restricted paths. <p> To do this, we will use the `pytrax` python module https://doi.org/10.1016/j.softx.2019.100277, https://github.com/PMEAL/pytrax in conjunction with some of the typical ancillary data analysis components. <p>
In this example, we neeed `matplotlib` for visualisation, `numpy` for mumerical analysis, `curve_fit` from `scipy.optimize` to determine the gradient of mean squared displacement with time step, `pytrax` as described above, and `glob` for filesystem operations. <p>
Also set some threshold values for the volumetric phases observable in our imagery...

In [1]:
import numpy as np
import nrrd as nrrd
from scipy.optimize import curve_fit
import pytrax as pt
import glob

por_lower    = 18504
precip_lower = 29524

Define a function to "configure" an image. Take the raw image path on disk, load to an array using `nrrd.read()` and then run logical array operations to return a thresholded image representing the geometry defined by the threshold values.

In [2]:
def conf_image(raw_image, por_lower=por_lower, precip_lower=precip_lower):
    image  = nrrd.read(raw_image)[0]                                        # Read the image file from disk
    porosity    = image < por_lower                                         # Segment the devoid porosity
    precipitate = np.logical_and(image < precip_lower, image > por_lower)   # Segment precipitate by threshold
    total       = image < precip_lower                                      # Segment everything by threshold
    return porosity, precipitate, total                                     # Return three segmented images

Define a second function to calculate the tau of an image. Create a `RandomWalk` object using the image passed to the function as `processed_image` with the number of time steps defined as `nt`, the number of walkers defined as `nw`, the start position defined as the same `True` or randomised `False` and a stride value. `num_proc` is self explanatory.<p>
Run the random walk operation with the previously provided parameters, and calculate the mean squared displacement `rw.calc_msd()`. Fit the values to a linear function $y = mx + c$, determine $m$, and the inverse of this is the tortuosity. Return the tortuosity. <p>
    
Also set some default parameters for the random walk. This ensures we use the same values every time, and keeps the code tidy. Set 20,000 walkers with 10,000 time points, distributed randomly throughout the structure, a step value of 1 and run this on 4 cores.

In [3]:
def calc_tau(processed_image, nt=1e4, nw=20000, same_start=False, stride=1, num_proc=4):
    rw = pt.RandomWalk(processed_image)
    rw.run(nt=nt, nw=nw, same_start=same_start, stride=stride, num_proc=num_proc)
    rw.calc_msd()
    y = rw.msd
    x = np.linspace(0, rw.nt, len(y))
    A = np.vstack([x, np.ones(len(x))]).T
    a, res, _, _ = np.linalg.lstsq(A, y, rcond=-1)
    return 1 / a[0]

Repeat scans are stored in folders `Raw Data/PCScan4_R1 ... R2 ... R3 ... etc`. Read the name of each folder into disk, and sort them numerically.

In [4]:
repeats = sorted(glob.glob("../In-situ CT/100PC_Ins/Raw Data/PCScan4_R*"))[:-2]

The pre-determined times at which we want calculations are provided in the `times` array in the following cell. We need precipitate tortuosity `precip_tau`, porosity tortuosity `por_tau`, and total tortuosity `tot_tau` values at each point.

In [5]:
times = np.array([530, 600, 1920, 2000, 3000, 3140, 4680, 5400, 5780, 
                  15360, 15540, 22440, 26040, 29640, 31440, 33240, 
                  35040, 36840, 40440, 44040, 47731, 49611])

precip_tau = np.zeros((len(times), len(repeats)))
por_tau    = np.zeros((len(times), len(repeats)))
tot_tau    = np.zeros((len(times), len(repeats)))

Loop through the repeats, and then through the time of each scan we want for our results, determine the tortuosity of each volumetric phase, and write this to the array(s) defined above. Print output back to the user.

In [6]:
for ii in range(0, len(repeats)):
    print("Repeat {} of {}...".format(ii+1, len(repeats)))
    for jj in range(0, len(times)):
        
        images = "{}\PCScan4_t{}.nrrd".format(repeats[ii], times[jj])
        porosity, precipitate, total = conf_image(images, por_lower, precip_lower)
        
        por_tau[jj,ii]    = calc_tau(porosity)
        precip_tau[jj,ii] = calc_tau(precipitate)
        tot_tau[jj,ii]    = calc_tau(total)
        
        print("t = {}, Por tau = {} Prec tau = {} Tot tau = {}".format(times[jj],
                                                                       np.round(por_tau[jj,ii],    3), 
                                                                       np.round(precip_tau[jj,ii], 3), 
                                                                       np.round(tot_tau[jj,ii],    3)))
    print("")

Repeat 1 of 4...
t = 530, Por tau = 3.591 Prec tau = 1.774 Tot tau = 1.355
t = 600, Por tau = 1.962 Prec tau = 2.275 Tot tau = 1.265
t = 1920, Por tau = 2.478 Prec tau = 1.909 Tot tau = 1.278
t = 2000, Por tau = 2.141 Prec tau = 2.057 Tot tau = 1.248
t = 3000, Por tau = 2.497 Prec tau = 1.919 Tot tau = 1.286
t = 3140, Por tau = 2.511 Prec tau = 1.923 Tot tau = 1.287
t = 4680, Por tau = 6.881 Prec tau = 1.835 Tot tau = 1.666
t = 5400, Por tau = 8.154 Prec tau = 2.018 Tot tau = 1.856
t = 5780, Por tau = 5.907 Prec tau = 1.902 Tot tau = 1.653
t = 15360, Por tau = 4.764 Prec tau = 1.81 Tot tau = 1.491
t = 15540, Por tau = 5.605 Prec tau = 1.901 Tot tau = 1.603
t = 22440, Por tau = 9.39 Prec tau = 2.196 Tot tau = 1.96
t = 26040, Por tau = 8.666 Prec tau = 2.122 Tot tau = 1.912
t = 29640, Por tau = 7.802 Prec tau = 2.145 Tot tau = 1.893
t = 31440, Por tau = 8.119 Prec tau = 2.166 Tot tau = 1.913
t = 33240, Por tau = 7.527 Prec tau = 2.083 Tot tau = 1.849
t = 35040, Por tau = 7.991 Prec tau =